In [1]:
#Dependencies
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# Create a reference the CSV file desired
csv_path = "FARS2016NationalCSV/accident.csv"

# Read the CSV into a Pandas DataFrame
accidents = pd.read_csv(csv_path)


In [3]:
accidents_df = accidents.drop(accidents.columns[[1,2,4,5,6,8,17,18,19,20,24,25,26,27,30,31,32,33,39,40,41,42,43,44,45,46]], axis=1)

In [4]:
csv_path1 = "FARS2016NationalCSV/states.csv"
states = pd.read_csv(csv_path1)

In [5]:
accidents_df['STATE'] = accidents_df['STATE'].map(states.set_index('Codes')['State'])

In [6]:
xls_path = "GLC.xlsx"
xl = pd.ExcelFile(xls_path)
state_codes = xl.parse("Sheet 1")
state_codes.head()

,Territory,State Name,State Abbreviation,State Code,City Name/County Name,City Code,County Code
0,U,ALABAMA,AL,1.0,ABBEVILLE,10.0,67.0
1,U,ALABAMA,AL,1.0,ALBERTVILLE,50.0,95.0
2,U,ALABAMA,AL,1.0,ALEXANDER CITY,60.0,123.0
3,U,ALABAMA,AL,1.0,ANDALUSIA,90.0,39.0
4,U,ALABAMA,AL,1.0,ANDERSON,96.0,77.0


In [7]:
state_codes.fillna(0, inplace=True)

In [8]:
state_codes["City Code"]= state_codes["City Code"].astype(int)
state_codes["County Code"]= state_codes["County Code"].astype(int)
state_codes["State Name"] = state_codes["State Name"].astype(str) 

In [9]:
state_codes = state_codes.apply(lambda x: x.astype(str).str.title())

In [10]:
state_codes["state_county_city"] = state_codes["State Name"].map(str) + "_" + state_codes["County Code"].map(str) + "_" + state_codes["City Code"].map(str)

In [11]:
accidents_df["state_county_city"] = accidents_df["STATE"].map(str) + "_" + accidents_df["COUNTY"].map(str) + "_" + accidents_df["CITY"].map(str)

In [12]:
state_codes.loc[state_codes.duplicated('state_county_city', keep=False)]

,Territory,State Name,State Abbreviation,State Code,City Name/County Name,City Code,County Code,state_county_city
1048,U,Alaska,Ak,2.0,Denali,0,68,Alaska_68_0
1049,U,Alaska,Ak,2.0,Denali Borough,0,68,Alaska_68_0
46087,0,0,0,0.0,0,0,0,0_0_0
46088,For Official Use Only.,0,0,0.0,0,0,0,0_0_0


In [13]:
state_codes = state_codes.drop(state_codes.index[[1049,46088]])


In [14]:
accidents_df['CITY'] = accidents_df['state_county_city'].map(state_codes.set_index('state_county_city')['City Name/County Name'])

In [15]:
accidents_df.head()

,STATE,VE_FORMS,PERMVIT,COUNTY,CITY,DAY,MONTH,YEAR,DAY_WEEK,HOUR,...,LGT_COND,WEATHER1,WEATHER2,WEATHER,CF1,CF2,CF3,FATALS,DRUNK_DR,state_county_city
0,Alabama,1,1,73,Hoover,1,1,2016,6,9,...,1,1,0,1,0,0,0,1,1,Alabama_73_1716
1,Alabama,1,2,73,Hoover,10,1,2016,1,19,...,3,1,0,1,0,0,0,1,1,Alabama_73_1716
2,Alabama,1,1,73,Jefferson,1,1,2016,6,2,...,3,10,0,10,0,0,0,1,0,Alabama_73_0
3,Alabama,1,1,73,Jefferson,17,1,2016,1,21,...,3,1,0,1,0,0,0,1,1,Alabama_73_0
4,Alabama,1,1,73,Vestavia Hills,26,1,2016,3,20,...,3,2,0,2,0,0,0,1,0,Alabama_73_3125


In [16]:
csv_path2 = "FARS2016NationalCSV/weather.csv"
csv_path3 = "FARS2016NationalCSV/CF.csv"
csv_path4 = "FARS2016NationalCSV/light.csv"
csv_path5 = "FARS2016NationalCSV/road.csv"
weather = pd.read_csv(csv_path2)
cf = pd.read_csv(csv_path3)
light_cond = pd.read_csv(csv_path4)
road = pd.read_csv(csv_path5)

In [17]:
accidents_df['WEATHER'] = accidents_df['WEATHER'].map(weather.set_index('Code')['Weather'])
accidents_df['WEATHER1'] = accidents_df['WEATHER1'].map(weather.set_index('Code')['Weather'])
accidents_df['WEATHER2'] = accidents_df['WEATHER2'].map(weather.set_index('Code')['Weather'])

In [18]:
accidents_df['CF1'] = accidents_df['CF1'].map(cf.set_index('Code')['Attribute'])
accidents_df['CF2'] = accidents_df['CF2'].map(cf.set_index('Code')['Attribute'])
accidents_df['CF3'] = accidents_df['CF3'].map(cf.set_index('Code')['Attribute'])

In [19]:
accidents_df['LGT_COND'] = accidents_df['LGT_COND'].map(light_cond.set_index('Code')['Attribute'])

In [20]:
accidents_df['REL_ROAD'] = accidents_df['REL_ROAD'].map(road.set_index('Code')['Attribute'])

In [21]:
accidents_df = accidents_df.drop(accidents_df.columns[[3,26]], axis=1)

In [22]:
accidents_df


,STATE,VE_FORMS,PERMVIT,CITY,DAY,MONTH,YEAR,DAY_WEEK,HOUR,MINUTE,...,REL_ROAD,LGT_COND,WEATHER1,WEATHER2,WEATHER,CF1,CF2,CF3,FATALS,DRUNK_DR
0,Alabama,1,1,Hoover,1,1,2016,6,9,25,...,Outside Trafficway,Daylight,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
1,Alabama,1,2,Hoover,10,1,2016,1,19,59,...,On Roadside,Dark - Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
2,Alabama,1,1,Jefferson,1,1,2016,6,2,20,...,On Roadside,Dark - Lighted,Cloudy,No Additional Atmospheric Conditions,Cloudy,None,None,None,1,0
3,Alabama,1,1,Jefferson,17,1,2016,1,21,45,...,On Median,Dark - Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
4,Alabama,1,1,Vestavia Hills,26,1,2016,3,20,7,...,Outside Trafficway,Dark - Lighted,Rain,No Additional Atmospheric Conditions,Rain,None,None,None,1,0
5,Alabama,1,1,Pike,1,1,2016,6,1,10,...,On Roadside,Dark - Not Lighted,Cloudy,No Additional Atmospheric Conditions,Cloudy,None,None,None,1,0
6,Alabama,1,1,Calera,1,1,2016,6,23,34,...,On Roadside,Dark - Not Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,0
7,Alabama,1,1,Limestone,4,1,2016,2,17,30,...,On Roadside,Dark - Not Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
8,Alabama,2,4,Priceville,4,1,2016,2,5,37,...,On Roadway,Dark - Not Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,2,0
9,Alabama,1,2,Houston,5,1,2016,3,15,18,...,On Roadside,Daylight,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,2,1


In [23]:
accidents_df.columns

Index(['STATE', 'VE_FORMS', 'PERMVIT', 'CITY', 'DAY', 'MONTH', 'YEAR',
       'DAY_WEEK', 'HOUR', 'MINUTE', 'ROUTE', 'TWAY_ID', 'TWAY_ID2', 'HARM_EV',
       'MAN_COLL', 'REL_ROAD', 'LGT_COND', 'WEATHER1', 'WEATHER2', 'WEATHER',
       'CF1', 'CF2', 'CF3', 'FATALS', 'DRUNK_DR'],
      dtype='object')

In [24]:
# create a dictionary of values with their corresponding description for the column MAN_COLL
man_col_dict = {0 : "Not a Collision with a Motor Vehicle In-Transport",
            1 : "Front-to-Rear", 
            2 : "Front-to-Front",
            6:"Angle",
            7:"Sideswipe-Same Direction",
            8: "Sideswipe-Opposite Direction",
            9: "Rear-to-Side",
            10: "Rear-to-Rear",
            11: "Other",
            98: "Not Reported",
            99: "Unknown"          
          }

# replace the numbers in MAN_COLL column with the corresponding description from the dictionary
accidents_df['MAN_COLL'] = [man_col_dict[val] for val in accidents_df['MAN_COLL']]

In [25]:
# create a dictionary of values with their corresponding description for the column HARM_EV

harm_ev_dict = {1 : "Rollover/Overturn",
                2 : "Fire/Explosion",
                3: "Immersion or Partial Immersion",
                4: "Gas Inhalation",
                51: "Jackknife (harmful to this vehicle)",
                6: "Injured in Vehicle (Non-Collision)",
                44: "Pavement Surface Irregularity (Ruts, Potholes, Grates, etc.)",
                7: "Other Non-Collision",
                16: "Thrown or Falling Object",
                72: "Cargo/Equipment Loss or Shift (harmful to this vehicle)",
                5: "Fell/Jumped from Vehicle",
                12: "Motor Vehicle In-Transport",
                54: "Motor Vehicle In-Transport Strikes",
                55: "Motor Vehicle in Motion Outside the Trafficway",
                8: "Pedestrian",
                9: "Pedalcyclist",
                10: "Railway Vehicle",
                74: "Road Vehicle on Rails",
                11: "Live Animal",
                49: "Ridden Animal",
                18: "Other Object (Not Fixed)",
                15: "Non-Motorist on Personal Conveyance",
                14: "Parked Motor Vehicle",
                45: "Working Motor Vehicle",
                73: "Object That Had Fallenfrom Motor Vehicle In-Transport",
                17: "Boulder",
                19: "Building",
                58: "Ground",
                20: "Impact Attenuator/Crash Cushion",
                50: "Bridge Overhead Structure",
                21: "Bridge Pier or Support",
                23: "Bridge Rail",
                24: "Guardrail Face",
                52: "Guardrail End",
                25: "Concrete Traffic Barrier",
                57: "Cable Barrier",
                26: "Other Traffic Barrier",
                59: "Traffic Sign Support",
                46: "Traffic Signal Support",
                30: "Utility Pole/Light Support",
                31: "Other Post, Other Pole, or Other Supports",
                32: "Culvert",
                33: "Curb",
                34: "Ditch",
                35: "Embankment",
                38: "Fence",
                39: "Wall",
                40: "Fire Hydrant",
                41: "Shrubbery",
                42: "Tree",
                48: "Snow Bank",
                53: "Mail Box",
                43: "Other Fixed Object",
                98: "Not Reported",
                99: "Unknown" 
               }
# replace the numbers in HARM_EV column with the corresponding description from the dictionary
accidents_df['HARM_EV'] = [harm_ev_dict[val] for val in accidents_df['HARM_EV']]

In [26]:
# create a dictionary of values with their corresponding description for the column ROUTE
route_dict = {  1: "Interstate",
                2: "U.S. Highway",
                3: "State Highway",
                4: "County Road",
                5: "Local Street - Township",
                6: "Local Street - Municipality",
                7: "Local Street - Frontage Road",
                8: "Other",
                9: "Unknown"
            }
# replace the numbers in ROUTE column with the corresponding description from the dictionary
accidents_df['ROUTE'] = [route_dict[val] for val in accidents_df['ROUTE']]

In [27]:
# visualize the data frame
accidents_df

,STATE,VE_FORMS,PERMVIT,CITY,DAY,MONTH,YEAR,DAY_WEEK,HOUR,MINUTE,...,REL_ROAD,LGT_COND,WEATHER1,WEATHER2,WEATHER,CF1,CF2,CF3,FATALS,DRUNK_DR
0,Alabama,1,1,Hoover,1,1,2016,6,9,25,...,Outside Trafficway,Daylight,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
1,Alabama,1,2,Hoover,10,1,2016,1,19,59,...,On Roadside,Dark - Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
2,Alabama,1,1,Jefferson,1,1,2016,6,2,20,...,On Roadside,Dark - Lighted,Cloudy,No Additional Atmospheric Conditions,Cloudy,None,None,None,1,0
3,Alabama,1,1,Jefferson,17,1,2016,1,21,45,...,On Median,Dark - Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
4,Alabama,1,1,Vestavia Hills,26,1,2016,3,20,7,...,Outside Trafficway,Dark - Lighted,Rain,No Additional Atmospheric Conditions,Rain,None,None,None,1,0
5,Alabama,1,1,Pike,1,1,2016,6,1,10,...,On Roadside,Dark - Not Lighted,Cloudy,No Additional Atmospheric Conditions,Cloudy,None,None,None,1,0
6,Alabama,1,1,Calera,1,1,2016,6,23,34,...,On Roadside,Dark - Not Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,0
7,Alabama,1,1,Limestone,4,1,2016,2,17,30,...,On Roadside,Dark - Not Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,1,1
8,Alabama,2,4,Priceville,4,1,2016,2,5,37,...,On Roadway,Dark - Not Lighted,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,2,0
9,Alabama,1,2,Houston,5,1,2016,3,15,18,...,On Roadside,Daylight,Clear,No Additional Atmospheric Conditions,Clear,None,None,None,2,1


In [28]:
# check if df consists of any null values
accidents_df.isnull().sum()

STATE           0
VE_FORMS        0
PERMVIT         0
CITY          247
DAY             0
MONTH           0
YEAR            0
DAY_WEEK        0
HOUR            0
MINUTE          0
ROUTE           0
TWAY_ID         0
TWAY_ID2    25283
HARM_EV         0
MAN_COLL        0
REL_ROAD        0
LGT_COND        0
WEATHER1        0
WEATHER2        0
WEATHER         0
CF1             0
CF2             0
CF3             0
FATALS          0
DRUNK_DR        0
dtype: int64

In [35]:
accidents_df.to_csv("accidents-final.csv")